<a href="https://colab.research.google.com/github/dvsseed/AR2/blob/master/PyTorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [PyTorch]—MNIST @Date:2020-08-07

---



神經網路的基本訓練流程：

1. 定義神經網路、並設置學習的參數或權重
2. 迭代輸入 Training data
3. 設定好的神經網路開始處理輸入資料
4. 計算 Loss function (損失函數)，即為計算『output 與正確答案的差距』
5. 藉由 Loss function 得到應該更改的權重
6. 更新神經網路的權重，通常使用較簡單的規則，比如：新的權重=舊的權重–(Learning rate學習率*gradient梯度)
7. 儲存模型參數

In [ ]:
# -*- coding: utf-8 -*-
# 匯入套件
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dset
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
# 確認 GPU 是否可用
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [ ]:
# 圖片的轉換(Transform)
transform = transforms.Compose(
    [
     transforms.ToTensor(),  # 轉成 Tensor 型態
     transforms.Normalize((0.5,), (0.5,)),  # 其正規化
    ]
)

In [ ]:
# 資料集(Dataset)
trainSet = datasets.MNIST(root='MNIST', download=True, train=True, transform=transform)
testSet = datasets.MNIST(root='MNIST', download=True, train=False, transform=transform)
trainLoader = dset.DataLoader(trainSet, batch_size=64, shuffle=True)
testLoader = dset.DataLoader(testSet, batch_size=64, shuffle=False)

Extracting MNIST/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/MNIST/raw


Extracting MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/MNIST/raw



Extracting MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/MNIST/raw


Extracting MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# CNN模型(Model)的架構
class Net(nn.Module):  # 使用 torch.nn 來建構神經網路
    def __init__(self):
        super(Net, self).__init__()
        # input shape (1, 28, 28)
        self.conv1 = nn.Conv2d(in_channels=1,  # input height
                    out_channels=32,  # n_filters
                    kernel_size=5,   # filter size
                    stride=1,     # filter movement/step
                    padding=2     # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
                    )  # output shape (32, 28, 28)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=64*7*7, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)  # fully connected layer, output 10 classes

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))  # activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), kernel_size=2, stride=2))
        x = x.view(-1, 64*7*7)  # flatten the output of conv2 to (batch_size, 64*7*7)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net().to(device)
print(net)  # 列印結構


Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
params = list(net.parameters())
print(len(params))  # 列印參數量

for n in range(len(params)):
    print(params[n].size())

10
torch.Size([32, 1, 5, 5])
torch.Size([32])
torch.Size([64, 32, 5, 5])
torch.Size([64])
torch.Size([128, 3136])
torch.Size([128])
torch.Size([64, 128])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [ ]:
# 參數(Parameters)設定
epochs = 3  # 訓練的迭代次數
lr = 0.001  # Learning rate，反向傳播的學習率
criterion = nn.CrossEntropyLoss()  # 損失函數(Loss function)
# criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)  # optimize all cnn parameters
# optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  # 優化器=隨機梯度下降(SGD)，momentum為同方向的梯度更新幅度越來越大，反方向的則變小，通常都設0.9

In [ ]:
# 訓練(Train)模型
print('Training Started.')
for epoch in range(epochs):
    running_loss = 0.0

    for times, data in enumerate(trainLoader):
        inputs, labels = data[0].to(device), data[1].to(device)
        # inputs = inputs.view(inputs.shape[0], -1)  # 將inputs的維度壓到符合模型的輸入(input=784)

        # 反向傳播(Backward propagation)，通過計算預測結果與正解的距離（即loss function），來反向地更新模型的權重，使模型越來越準確
        # Zero the parameter gradients
        optimizer.zero_grad()  # 將參數、梯度緩衝區歸零

        # Foward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if times % 100 == 99 or times+1 == len(trainLoader):
            print('Epoch: [%d/%d, %d/%d], loss: %.3f' % (epoch+1, epochs, times+1, len(trainLoader), running_loss/2000))

print('Training Finished.')

Training Started.
Epoch: [1/3, 100/938], loss: 0.078
Epoch: [1/3, 200/938], loss: 0.107
Epoch: [1/3, 300/938], loss: 0.127
Epoch: [1/3, 400/938], loss: 0.143
Epoch: [1/3, 500/938], loss: 0.156
Epoch: [1/3, 600/938], loss: 0.168
Epoch: [1/3, 700/938], loss: 0.179
Epoch: [1/3, 800/938], loss: 0.189
Epoch: [1/3, 900/938], loss: 0.199
Epoch: [1/3, 938/938], loss: 0.202
Epoch: [2/3, 100/938], loss: 0.009
Epoch: [2/3, 200/938], loss: 0.017
Epoch: [2/3, 300/938], loss: 0.025
Epoch: [2/3, 400/938], loss: 0.034
Epoch: [2/3, 500/938], loss: 0.042
Epoch: [2/3, 600/938], loss: 0.050
Epoch: [2/3, 700/938], loss: 0.058
Epoch: [2/3, 800/938], loss: 0.065
Epoch: [2/3, 900/938], loss: 0.073
Epoch: [2/3, 938/938], loss: 0.076
Epoch: [3/3, 100/938], loss: 0.006
Epoch: [3/3, 200/938], loss: 0.013
Epoch: [3/3, 300/938], loss: 0.019
Epoch: [3/3, 400/938], loss: 0.025
Epoch: [3/3, 500/938], loss: 0.031
Epoch: [3/3, 600/938], loss: 0.038
Epoch: [3/3, 700/938], loss: 0.045
Epoch: [3/3, 800/938], loss: 0.050
Ep

In [ ]:
# 測試(Test)
correct = 0
total = 0

with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # inputs = inputs.view(inputs.shape[0], -1)

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100*correct / total))

class_correct = [0 for i in range(10)]
class_total = [0 for i in range(10)]

with torch.no_grad():
    for data in testLoader:
        inputs, labels = data[0].to(device), data[1].to(device)
        # inputs = inputs.view(inputs.shape[0], -1)

        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            # print(class_correct)
            # print(class_total)


for i in range(10):
    print('Accuracy of %d: %3f' % (i, (class_correct[i] / class_total[i])))

Accuracy of the network on the 10000 test images: 97 %
Accuracy of 0: 0.986842
Accuracy of 1: 0.983784
Accuracy of 2: 0.994186
Accuracy of 3: 0.961538
Accuracy of 4: 0.983051
Accuracy of 5: 0.984127
Accuracy of 6: 0.968750
Accuracy of 7: 0.939024
Accuracy of 8: 0.965035
Accuracy of 9: 0.982036


In [ ]:
# 儲存模型
torch.save(net, 'cnn.pth')